# <font color="#76b900">**Notebook 99:** Deploying Your First NIM</font>


In this notebook, we will be loading in a NIM model to run on your GPU-enabled environment. The process is documented extensively in the [NIM "Getting Started" documentation](https://docs.nvidia.com/nim/large-language-models/latest/getting-started.html), so feel free to refer to it if you need any more details. 

<hr>

### **NOTE:** THIS IS ONLY FOR REFERENCE!
**(It also will not work in this environment since it assumes host docker access)**

<hr>

NIMs are packaged as container images on a per model/model family basis. These containers include a runtime that runs on any NVIDIA GPU with sufficient GPU memory, but some model/GPU combinations like this one are especially well-optimized.

### **Kickstarting On Application Startup**

One way to kickstart the environment alongside other microservices is to specify it in the docker-compose file as shown below and demonstrated in [`composer/docker-compose.yml`](./composer/docker-compose.yml):

```sh
  nim:
    ## VLLM-backed LLM NIM: Provides speedup while also being easy to work with
    image: nvcr.io/nim/meta/llama3-8b-instruct:1.0.3  ## <-the image you're working with
    entrypoint: /bin/sh -c "echo 'nim image downloaded'"
    deploy:
      resources:
        reservations:
          devices:
            - driver: nvidia
              count: 1
              capabilities: [gpu]
    ports:
      - 8000:8000
    entrypoint: env
    environment:
      NGC_API_KEY: ${NGC_API_KEY}  ## <- needs to be specified here or in env file
```

The one catch is that we have yet to provide the NGC_API_KEY, which is used internally inside the NIM image to help download the right model. To kickstart the microservice, please sign up for an NGC account if you do not already have one. Once there, navigate to [**`Setup Menu`**](https://org.ngc.nvidia.com/setup) and generate your API key in **`Generate API Key`**. This should give an API key which is used to connect to the NGC model registry.

In [ ]:
## TODO: Add your NGC API KEY ()
## NOTE: It should NOT start with `nvapi-`. That's for NVCF & build.nvidia.com endpoints.
%env NGC_API_KEY=...

In [ ]:
%%bash
echo "$NGC_API_KEY" | docker login nvcr.io --username "\$oauthtoken" --password-stdin

<br>

After this, you should be able to select your models of choice from the [**NGC Catalog**](https://catalog.ngc.nvidia.com/containers?filters=nvidia_nim), though in this course we will be defaulting to [**Llama-3-8B**](https://catalog.ngc.nvidia.com/orgs/nim/teams/meta/containers/llama3-8b-instruct). The code block below specifies the model of interest and then proceeds to actually download the NIM, download the associated model, and kickstart the microservice with docker.

In [ ]:
# Choose a LLM NIM Image from NGC
%env IMG_NAME=nvcr.io/nim/meta/llama3-8b-instruct:1.0.3

# Choose a path on your system to cache the downloaded models
%env LOCAL_NIM_CACHE=./cache/nim

!mkdir -p "$LOCAL_NIM_CACHE"

## Start the LLM NIM
# !docker rm -f nim
!docker run --rm --name=nim \
    --runtime=nvidia \
    --gpus all \
    --shm-size=16GB \
    --network nvidia-sizing \
    -e NGC_API_KEY=$NGC_API_KEY \
    -v $LOCAL_NIM_CACHE:/opt/nim/.cache \
    -u $(id -u) \
    -p 8000:8000 \
    $IMG_NAME

## NOTES:
## - gpus, runtime, and shared-memory-size all help specify which resources 
##     and environments the service can leverage. 
## - Since we put our microservices (including our jupyter labs instance)
##     on the nvidia-sizing network in the dockerfile, specifying this helps
##     the services play nicely together. This detail is unimportant.
## - -e specifies environment variables, and NIMs want access to NGC.
## - -v specifies volume mounts, and we may want access to the NIM model cache
##     to avoid redownloading resources and maybe modify some settings.
## - -u specifies user profile and -p specifies port mapping. We usually 
##     want parity with our other services to keep things consistent.
## - $IMG_NAME AKA nvcr.io/nim/<model-name> here is the image we run

<br><hr>

#### **WHEN YOU SEE THE FOLLOWING, YOU CAN ASSUME THE SERVICE IS RUNNING:**
```json
INFO ... on.py:62] Application startup complete.
INFO ... server.py:214] Uvicorn running on http://0.0.0.0:8000 (Press CTRL+C to quit)
INFO...8 metrics.py:334] Avg prompt throughput: 0.3 tokens/s, Avg generation throughput: 1.5 tokens/s, Running: 0 reqs, Swapped: 0 reqs, Pending: 0 reqs, GPU KV cache usage: 0.0%, CPU KV cache usage: 0.0%
```